# CoT (Chain of Thought)

In [62]:
from dotenv import load_dotenv
load_dotenv()

True

In [63]:
import openai
client = openai.OpenAI()

### 일반 프롬프트 vs Zero-shot CoT

In [64]:
# 일반 프롬프트
prompt = '123 x 31 + 7은 얼마인가요?'

# Zero-shot CoT 적용 프롬프트
cot_prompt = '단계별로 계산해 보세요: 123 x 31 + 7은 얼마인가요?'

res = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{"role": "user", "content": prompt}]
)

cot_res = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{"role": "user", "content": cot_prompt}]
)

In [65]:
print(res.choices[0].message.content)

123 x 31 = 3813

3813 + 7 = 3820

따라서 123 x 31 + 7는 3820입니다.


In [66]:
print(cot_res.choices[0].message.content)

먼저 \[ 123 \times 31 \]을 계산하면, 3813이 됩니다.
그런 다음, 3813에 7을 더하면, \[3813 + 7 = 3820\]이 됩니다.

따라서, 123 x 31 + 7은 3820입니다.


### Few-shot CoT

In [67]:
few_shot_cot = """
Q: 123 x 31 은 얼마인가요?
A: 
1. 123를 분해합니다: 123 = 100 + 20 + 3
2. 각 항을 31과 곱합니다:
   - 100 x 31 = 3100
   - 20 x 31 = 620
   - 3 x 31 = 93

3. 이제 이 결과들을 더합니다:
   - 3100 + 620 + 93

4. 계산을 진행하면:
   - 3100 + 620 = 3720
   - 3720 + 93 = 3813 

따라서, 123 x 31 = 3813입니다.

Q: 789 x 56 은 얼마인가요?
A:
"""

res = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{"role": "user", "content": few_shot_cot}]
)

In [68]:
print(res.choices[0].message.content)

1. 789를 분해합니다: 789 = 700 + 80 + 9
2. 각 항을 56과 곱합니다:
   - 700 x 56 = 39200
   - 80 x 56 = 4480
   - 9 x 56 = 504

3. 이제 이 결과들을 더합니다:
   - 39200 + 4480 + 504

4. 계산을 진행하면:
   - 39200 + 4480 = 43680
   - 43680 + 504 = 44184

따라서, 789 x 56 = 44184입니다.


### 논리 문제 해결

In [77]:
prompt = "앨리스(Alice)는 밥(Bob)보다 나이가 많다. 밥(Bob)은 찰리(Charlie)보다 나이가 많다. 데이비드(David)는 앨리스(Alice)보다 어리지만, 에밀리(Emily)보다는 많다. 찰리(Charlie)는 프랭크(Frank)보다 많다. 에밀리(Emily)는 찰리(Charlie)보다 나이가 많지만, 밥(Bob)보다는 어리다. 프랭크(Frank)는 데이비드(David)보다 어리다. 그렇다면 가장 어린 사람은 누구인가요?"

cot_prompt = "단계적으로 생각한 과정을 보여주고, 단계적으로 생각한 결과로 대답해주세요: 앨리스(Alice)는 밥(Bob)보다 나이가 많다. 밥(Bob)은 찰리(Charlie)보다 나이가 많다. 데이비드(David)는 앨리스(Alice)보다 어리지만, 에밀리(Emily)보다는 많다. 찰리(Charlie)는 프랭크(Frank)보다 많다. 에밀리(Emily)는 찰리(Charlie)보다 나이가 많지만, 밥(Bob)보다는 어리다. 프랭크(Frank)는 데이비드(David)보다 어리다. 그렇다면 가장 어린 사람은 누구인가요?"

In [78]:
res = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{"role": "user", "content": prompt}]
)

cot_res = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{"role": "user", "content": cot_prompt}]
)

In [79]:
print(res.choices[0].message.content)

에밀리(Emily)입니다.


In [80]:
print(cot_res.choices[0].message.content)

1. 앨리스(Alice) > 밥(Bob)
2. 밥(Bob) > 찰리(Charlie)
3. 데이비드(David) < 앨리스(Alice) , 데이비드(David) > 에밀리(Emily)
4. 찰리(Charlie) > 프랭크(Frank)
5. 에밀리(Emily) > 찰리(Charlie) , 에밀리(Emily) < 밥(Bob)
6. 프랭크(Frank) < 데이비드(David)

결과: 데이비드(David) > 에밀리(Emily) > 찰리(Charlie) > 밥(Bob) > 앨리스(Alice) > 프랭크(Frank)

가장 어린 사람은 프랭크(Frank) 입니다.


### 코드 디버깅

In [73]:
prompt = """
다음과 같이 코드를 작성했더니 에러가 발생합니다. 왜일까요?

def add_numbers(a, b):
return a + b

print(add_numbers(1, 2))
"""

cot_prompt = """
다음과 같이 코드를 작성했더니 에러가 발생합니다. 
실수가 무엇인지 찾아 단계별로 설명해 주세요.

def add_numbers(a, b):
return a + b

print(add_numbers(1, 2))
"""

In [74]:
res = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{"role": "user", "content": prompt}]
)

cot_res = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{"role": "user", "content": cot_prompt}]
)

In [75]:
print(res.choices[0].message.content)

주어진 코드에서 들여쓰기가 올바르게 되어 있지 않기 때문에 에러가 발생합니다. 

함수 정의 시 함수 내부 코드들은 들여쓰기로 구분되어야 하지만, `return`문이 제대로 들여쓰기가 되어있지 않아서 인터프리터가 문맥을 이해하지 못하고 에러를 발생시킵니다.

올바르게 수정된 코드는 다음과 같습니다.

```python
def add_numbers(a, b):
    return a + b

print(add_numbers(1, 2))
```


In [76]:
print(cot_res.choices[0].message.content)

에러 발생 원인:
들여쓰기 오류
세부 설명:
1. 함수 정의부에서 코드 블록이 올바르게 들여써지지 않았습니다. Python은 함수나 조건문, 반복문 등 코드 블록이 시작될 때 콜론(:) 뒤에 들여쓰기를 요구합니다. 
2. 따라서 def add_numbers(a, b): 다음 줄에 있는 return a + b가 함수 add_numbers의 코드 블록에 속해야 합니다. 그러나 현재 코드에서는 들여쓰기가 되어 있지 않아 함수 내의 코드가 아니라고 인식됩니다.
3. 이 문제를 해결하기 위해서는 return a + b를 함수 add_numbers 내부로 들여써주어야 합니다.
해결 방법:
```python
def add_numbers(a, b):
    return a + b

print(add_numbers(1, 2))
```
